# Jupyter Notebooks - solutions to exercises

## Lesson 1 - introduction to jupyter

### Exercise 1.3

In [ ]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline


# gaussian function
def gauss(x,param):
    [a,b,c] = param
    return a*np.exp(-b*(x-c)**2)

# gaussian array y in interval -5<x-5 
nx = 100
x = np.linspace(-5.,5.,nx)
p = [2.0,0.5,1.5] # some parameters
y = gauss(x,p)

# add some noise
noise = np.random.normal(0,0.2,nx)
y += noise

@interact
def fit(n=(3,30)):
    pfit = np.polyfit(x,y,n)
    yfit = np.polyval(pfit,x)
    plt.plot(x,y,"r",label="Data")
    plt.plot(x,yfit,"b",label="Fit")
    plt.legend()
    plt.ylim(-0.5,2.5)
    plt.show()

### Exercise 1.4

In [ ]:
# %load random_walk.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def step():
    import random
    return 1. if random.random() > .5 else -1.

In [ ]:
def walk(n):
    x = np.zeros(n)
    dx = 1. / n
    for i in range(n - 1):
        x_new = x[i] + dx * step()
        if x_new > 5e-3:
            x[i + 1] = 0.
        else:
            x[i + 1] = x_new
    return x

In [ ]:
n = 100000
x = walk(n)

In [ ]:
plt.plot(x);

In [ ]:
t1 = %timeit -o walk(n)

In [ ]:
t1.best

In [ ]:
%%prun
walk(n)

## Lesson 2 - data analysis

###  *Exercise 2.1* 


In [ ]:
nobels_by_country_of_death = nobel.groupby('diedCountry',sort=True)["number"].sum()
nobels_by_country_of_death.sort_values(ascending=False).head(5)

###  *Exercise 2.2* 

In [ ]:
born_but_not_died_in_sweden = nobel[(nobel["bornCountryCode"] == "SE") 
                                    & (nobel["diedCountryCode"] != "SE") 
                                    ]
born_but_not_died_in_sweden.dropna(subset=['diedCountryCode'])

### Exercise 2.3

In [ ]:
toppings_tidy = pd.melt(
                        person_toppings, 
                        id_vars=["Person", "Pizza"], 
                        value_vars=["Ingredient 1", "Ingredient 2"], 
                        var_name="Ingredient priority", 
                        value_name="Ingredient"
                        )
toppings_tidy[toppings_tidy["Ingredient"] == "Ham"]

## Lesson 3 - accelerating python

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

### Exercise 3.1

In [ ]:
# %load random_walk.py

In [ ]:
def step():
    return 1. if random.random() > .5 else -1.

In [ ]:
def walk(n):
    x = np.zeros(n)
    dx = 1. / n
    for i in range(n - 1):
        x_new = x[i] + dx * step()
        if x_new > 5e-3:
            x[i + 1] = 0.
        else:
            x[i + 1] = x_new
    return x

In [ ]:
n = 100000
x = walk(n)

In [ ]:
plt.plot(x);

In [ ]:
t1 = %timeit -o walk(n)

In [ ]:
t1.best

### Exercise 3.2

In [ ]:
def primes(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
t_py = %timeit -o p = primes(100)

In [ ]:
%load_ext Cython

Start with simplest cython

In [ ]:
%%cython -a
def primes_simplecython(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
t_cy0 = %timeit -o p = primes_simplecython(100)

Now for proper cythonization. Add annotation (`-a`) if you want!

In [ ]:
%%cython -a
def primes_cython(int kmax):  # The argument will be converted to int or raise a TypeError.
    cdef int n, k, i  # These variables are declared with C types.
    cdef int p[1000]  # Another C type
    result = []  # A Python type
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
t_cy = %timeit -o p = primes_cython(100)

Factor 20 in speedup

Finally, let's compare with just-in-time compilation with numba

In [ ]:
from numba import jit, vectorize, float64

In [ ]:
@jit
def primes_jit(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
t_jit = %timeit -o p = primes_jit(100)

In [ ]:
print(" Python: %.3E\n Simply Cython: %.3E\n Proper Cython: %.3E\n Numba-jit: %.3E"%(t_py.best,t_cy0.best,t_cy.best,t_jit.best))

Just-in-time compilation comes close to Cython

Notice that the slowest run took much much longer than the fastest.  
`cache=True` stores the compiled function in file-based cache and avoids re-compilation on re-running


In [ ]:
@jit(cache=True)
def primes_jit2(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
%%timeit
p = primes_jit2(100)